In [31]:
import boto3
from botocore.config import Config
import os
import time


In [25]:
# Could put credentials here, but is not safe
# Put in home directory $HOME/.aws/credentials
North_Virginia_EC2 = Config(region_name="us-east-1") # Define region (default is us-east-1)
NV_EC2 = boto3.client('ec2', config=North_Virginia_EC2) # make ec2 client

In [43]:
# Criar instância usando imagem AMI customizada
# (Criar máquina na mão, e configura-la, depois salvar imagem para usar aqui)
# Precisa ver como a máquina precisa de configurar o sec.group e passar o sec.group aqui
POSTGRES_AMI_ID = "ami-062339edf6f2e8b89"
POSTGRES_SECURITY_GROUP_ID = "sg-0ba0c4e57212c6ff0"

def offer_delete_db(client):
    filter=[
        {
            'Name': 'tag:DIP_AUTOMATION_BOTO',
            'Values': [
                'True',
            ]
        },
    ]
    current_machines = client.describe_instances(Filters=filter)
    ids = []
    for i in current_machines["Reservations"]:
        print(f"id encontrado: { i['Instances'][0]['InstanceId'] }")
        ids.append(i["Instances"][0]["InstanceId"])
    if len(ids) != 0:
        print("You seem to already have instances running this AMI, would you like to delete them? (y/n)")
        a = input()
        if a.strip().lower() not in ["y", "yes", ""]:
            print("Aborting...")
            return 1
    delete = client.terminate_instances(InstanceIds=ids)
    return 0

def create_db(client, ami, sec_group):
    
    # Apaga pre-existentes
    if offer_delete_db(client):
        return

    response = client.run_instances(
    BlockDeviceMappings=[{
                'DeviceName': '/dev/xvda',
                'Ebs': {
                    'DeleteOnTermination': False,
                    'VolumeSize': 240,
                    'VolumeType': 'gp2'
                },
            },
    ],
    ImageId=ami,
    InstanceType='t2.medium',
    MaxCount=1,
    MinCount=1,
    Monitoring={
        'Enabled': False
    },
    SecurityGroupIds=[
        sec_group,
    ],
    TagSpecifications=[
        {
            'ResourceType': 'instance',
            'Tags': [
                {
                    'Key': 'DIP_AUTOMATION_BOTO',
                    'Value': 'True',
                },
                {
                    'Key': 'Name',
                    'Value': 'postgres-db',
                },
            ],
        },
    ],)
    instanceID = response["Instances"][0]["InstanceId"]
    # Await creation
    running = False
    while not running:
        status = client.describe_instances(InstanceIds=[instanceID])["Reservations"][0]["Instances"][0]["State"]
        running = (status["Name"] == 'running')
        time.sleep(0.2) # No need to spam
    # Assign Elastic IP
    allocationID = "eipalloc-0303aad0424188ba3" # Elastic Static IP: 3.210.8.204
    response = NV_EC2.associate_address(AllocationId=allocationID, InstanceId=instanceID)
    return


In [15]:
Ohio_EC2 = Config(region_name="us-east-1") # Define region 2
O_EC2 = boto3.client('ec2', config=Ohio_EC2) # make ec2 client

In [16]:
DJANGO_LAUNCH_TEMPLATE_ID = "lt-0fbce85036a0bc068"
# DJANGO_AMI_ID = "ami-0f44416378e6304c7"
# DJANGO_SECURITY_GROUP_ID = "sg-05af7ec23322811ef"
LOAD_BALANCER_ELASTIC_IP_ID = "eipalloc-01534ba1f309a7991"

# Criar instância do ORM direitinho na mão,
# salvar imagem e usar ela + sec.group aqui.
# Isso precisa ser em outra cloud, por isso us-east-1
# def create_orm_instance(client, ami, sec_group):
#     pass

def create_load_balancer(client, ):
    response = client.create_load_balancer(
        AvailabilityZones=[
            'us-east-1','us-east-2',
        ],
        Listeners=[
            {
                'InstancePort': 8080,
                'InstanceProtocol': 'HTTP',
                'LoadBalancerPort': 80,
                'Protocol': 'HTTP',
            },
        ],
        LoadBalancerName='django-load-balancer',)
    return


In [36]:
create_db(NV_EC2, POSTGRES_AMI_ID, POSTGRES_SECURITY_GROUP_ID)

You seem to already have instances running this AMI, would you like to delete them? (y/n)


In [44]:
def cleanup(client, client_2):
    offer_delete_db(client)

cleanup(NV_EC2, None)

id encontrado: i-0756905ef83c743ec
You seem to already have instances running this AMI, would you like to delete them? (y/n)
